# Day 3 Part 1

Learning objectives
- Use NCBI e-utilities with biopython
- Use BLAST with the command line


## Section 01 - e-utilities with biopython 
---

You can access the NCBI databases using e-utilities. There is a standalone version of e-utilities you can use from the command line, but there are also ways to exectue e-utilities from within python. See section 9.6 in the biopython cookbook. 

Here, we will import e-utilities 'Entrez' module from the Bio package. You should put in your email address to tell them who you are. We first have to generate a search term and tell the program which database we would like to access.  This search strategy is often called a 'handle'. Here, we are querying the protein database (db="protein") for the accession number AKA62179, (id="AKA62179") and we would like to retrieve this in genbank format (rettype=gb) as text (retmode=text).  

In [ ]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"  # tell NCBI who you are 
handle = Entrez.efetch(db="protein", id="AKA62179", rettype="gb", retmode="text")

This 'handle' can be read by read function within the SeqIO module (we used this above for reading fasta files). Just like before, we should tell .read() what the format of the input object is. In this case, it is in genbank format. Now, all the information from genbank for the AKA62179 sequence is stored in `record`. 

In [ ]:
record = SeqIO.read(handle, "genbank")

# file = SeqIO.read("Myfasta.fasta", "fasta")

# As before, this sequence record has attributes associated with it such as 
# .id, .description, .seq
print(record.id)
print(record.description)
print(record.seq)

But since we retrieved the entire record of this sequence from the internet, we can also access other information stored as a dictionary in `record.annotations`. Let's see what information is stored in the keys of the dictionary

In [ ]:
record.annotations.keys()

In [ ]:
## print two annotations associated with this record. 
## Remember how to work with dictionaries?

record.annotations["source"]


Rarely will we want to retrieve only one sequence at a time. Thankfully it is possible to `efetch` multiple IDs by providing a list, here as `desired_ids`. 

In [ ]:
desired_ids  = ["B4S1U9","AKA62179"]
handle = Entrez.efetch(db="protein", id=desired_ids, rettype="gb", retmode="text")


Now the search results for all elements of the list are stored in `handle`. 

However, remember what we had to do when trying to read a fasta file with more than one entry? 

Which `SeqIO` function did we have to use? Fill in the '????' below.

In [ ]:
records = SeqIO.parse(handle, "genbank")

# Like above, let's collect the record for each of these accessions in a list
# For writing

seqList = []

for record in records:
    seqList.append(record)
        
SeqIO.write(seqList, "efetched_sequences.fasta", "fasta")

In [ ]:
# PRACTICE
# Do this exercice again using proteins of interest to you
# Write the efetched sequences in a new fasta file



In [ ]:
# PRACTICE

#  the file 'accessions.txt' is a list of accession numbers
#  With one accession number per line


# Use efetch to produce a fasta file of these accession numbers












### Section 02 - Protein Analysis with biopython
---

We are going to use another biopython tool called 'ProteinAnalysis'

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO

record = SeqIO.read("myFavouriteProtein_ncbi.fasta", "fasta")
sequence_str = str(record.seq)
analyze_this = ProteinAnalysis(sequence_str)

Here we stored the ProteinAnalysis of our sequence in 'analyze_this'.  Let's see what sort of analyses are available. Try the following:

In [ ]:
vars(analyze_this)

You will see that There are a couple elements that are already computed 
- Length
- sequence

But some entries like amino_acid_content/percentage are 'None'. 

We have to ask biopython to do those actions.  For example:

In [ ]:
analyze_this.count_amino_acids()

Store the counts of the amino acids in a dictionary called 'myAAcounts':

In [ ]:
myAAcounts = analyze_this.count_amino_acids()

In [ ]:
myAAfreq = analyze_this.get_amino_acids_percent()

Let's check how analyze_this looks like now!

In [ ]:
vars(analyze_this)

The `ProteinAnalysis` tool can also be used to calulate the aromaticity:

In [ ]:
analyze_this.aromaticity()

In [ ]:
vars(analyze_this)

In [ ]:
gravy = analyze_this.gravy() # Grand Average of Hydropathy

GRAVY is a measure of hydrophobicity, the more positive the number is, the more hydrophobic the protein. Let's write a code block to parse our NFU1 proteins and find the most hydrophobic protein. 

In [ ]:

def find_gravy(record):
    sequence_str = str(record.seq)
    analyze_this = ProteinAnalysis(sequence_str)
    return analyze_this.gravy()

# Empty variables
maxGravy = 0
maxGravyList = []
allGravy = []


#for loop to iterate over fasta file
for record in SeqIO.parse("NFU1_proteins.fasta", "fasta"):
    gravy = find_gravy(record)
    allGravy.append(gravy)
    if gravy >= maxGravy:
        # new winner!
        maxGravyList = []
        maxGravy = gravy
        maxGravyList.append(record)
print("Sequence %s had the highest gravy score of %s" % (record.id, str(maxGravy)))
print(maxGravyList)


### Install matplotlib

Type in the terminal in your environment: `conda install matplotlib`

Let's plot all the gravy scores in a histogram!


In [ ]:
#!conda install matplotlib

In [ ]:
import pylab
pylab.hist(allGravy,bins=20)

In [ ]:
## PRACTICE
# Repeat the exercise using another sequence that you will fetch from NCBI. 
handle = Entrez.efetch(db="protein", id="your protein of choise", rettype="gb", retmode="text")



## Section 03 - BLAST
---

### Installing BLAST locally 

Let's install BLAST locally using conda.  

Windows folks, please go here and download/install, keep track of where it installs in case we need to access the PATH: https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.14.1+-x64-win64.tar.gz

Follow these [instructions](https://2018-03-06-ibioic.readthedocs.io/en/latest/install_blast.html)


In [ ]:
# MacOSX and Linux, you can get the conda package
# type this line in a terminal (click on File > New > Terminal)
# conda install -c bioconda blast 

In [ ]:
# check that it is installed: 
!blastp -help

In [ ]:
%%bash
# make a new directory in our current directory
mkdir BLASTdb

#go into that directory
cd BLASTdb

#show available databases -- this will take a minute
update_blastdb.pl --showall

#download a small one: 
update_blastdb.pl --verbose --verbose --decompress swissprot 

In [ ]:
%%bash
pwd
# Check if the database is there?
cd BLASTdb/
ls -lhtr

In [ ]:
## DEFINE VARIABLES for the interactive python environment. 
%env blastQuery=blastQuery.fasta
%env db=BLASTdb/swissprot

In [ ]:
# This sets the variables so they are acccessible in the bash

In [ ]:
%%bash
#Spaces are important here: 
# bash would like the file name without quotes: 
# call the variables with a $
blastp -query $blastQuery -db $db -outfmt 6 -out $blastQuery.outfmt6.csv 

In [ ]:
# Look at the output file: 
!ls $blastQuery.outfmt6.csv
# Bash: 
!head $blastQuery.outfmt6.csv

The columns correspond to the following: 
> 'qaccver saccver pident length mismatch gapopen qstart qend sstart send evalue bitscore'

- qaccver = query ID
- saccver = hit ID
- pident = percent identity
- length = length
- mismatch = mismatch
- gapopen = gaps opened 
- qstart= query start
- qend = query end
- sstart = subject start
- send = subject end
- evalue = e-value (the lower the better!)
- bitscore = bit score (the higher the better!)


### BLAST with biopython

To blast with biopython: 

- put the swissprot database files (swissprot.\*) into a directory called 'BLASTdb'
- generate the command-lind (cline) for the BLASTP package as follows

In [ ]:
from Bio.Blast.Applications import NcbiblastpCommandline

blastp_cline = NcbiblastpCommandline(query="blastQuery.fasta", # query file (protein)
                                     db="BLASTdb/swissprot",  # path2database
                                     out="output.csv" , # output file - make i
                                     outfmt="6", # csv format
                                     evalue=0.001) # evalue cuutoff, 0.001 is fine
print(blastp_cline) # make sure it looks ok 


In [ ]:
#run your blast by calling blastp_cline()
blastp_cline()

In [ ]:
# PRACTICE

# Do this analyse again using your own protein

In [ ]:
# PRATICE

# Read the CSV file using python core and store the subject IDs of those sequences that are greater than 64% identical to your query sequence. 



Hints - in python core: 

- open the file line by line 
- uses the '.split()' function to split the line into a list 
- store variables for the positions you are interested in (i.e., subject ID)
  - for example, your query ID is going to be YOURLINE.split()[0] since it is the first cell on the line
- save a txt file with the subject IDs, each on their own line 
  - Q9UMS0.2
  - Q9QZ23.2 ... 

